In [ ]:
import os
import torch
from tqdm.notebook import tqdm
import pandas as pd
import glob

In [ ]:
from scripts import fgsm
from scripts.utils import open_img_as_tensor
from scripts.face_detectors import MediaPipe, YuNet, YoloFace
from scripts.facesegmentor import FaceSegementor
from scripts import image_attributes

In [ ]:
# Settings
image_attributes.save_color_images = False
image_attributes.save_lbp_images = False
image_attributes.save_gradient_images = False

In [ ]:
faceseg = FaceSegementor()

In [ ]:
yf = YoloFace()
yn = YuNet
mp = MediaPipe

In [ ]:
label_regions = ["bbox", "face"]
models = {
    "yf": YoloFace(),
    "yn": YuNet(),
    "mp": MediaPipe()
}

def get_label(label_model, label_region):
    return "e_" + label_region + "_" + label_model

Set the folders containing the images that you will include in the dataset

In [ ]:
folders = [r"./input"]

Create a dataset

In [ ]:
df_dataset = pd.DataFrame()
for folder in tqdm(folders, desc="All folders progress"):
    for file_path in tqdm(glob.glob(os.path.join(folder, "*.jpg")), desc="Current folder progress"):
        input_img = open_img_as_tensor(file_path)
        feats, grads, bboxes, masks = image_attributes.get_features(file_path, face_segmentor=faceseg)
        for label_region in label_regions:
            if label_region == "bbox":
                for i in range(len(masks)):
                    bbox_mask = masks[i]
                    bbox_mask[..., bboxes[i][1]:bboxes[i][3], bboxes[i][0]:bboxes[i][2]] = 1
                    masks[i] = bbox_mask
                
            for model in models:
                e_mins = []
                for data_grad, mask, bbox in zip(grads, masks, bboxes):    
                    display_img(mask)
                    e_mins.append(fgsm.binary_search(input_img, data_grad, models[model], mask, bbox))
                label_name = get_label(model, label_region)
                feats[label_name] = e_mins
    df_dataset = pd.concat([df_dataset, feats], ignore_index=True)

In [ ]:
df_dataset.to_csv("min_epsilon_dataset.csv")

___